## importing relevant libraries

In [1]:
from Crypto.Cipher import AES
import torch
from torch import nn
import pickle
import os
import socket
import struct

In [2]:
# setting key and nonce of 16 bytes
key = b"FedLwithHEandIoT"
nonce = b"FedLwithHEandIoT"

## Simple example

In [3]:
ciphertext = engine.encrypt(b"MyWeights")
print(ciphertext)

b'\xf85\xcd\xfa\xf4\x83%\xe4\xd5'


In [4]:
engine = AES.new(key, AES.MODE_EAX, nonce)

decryptedtext = engine.decrypt(ciphertext)
print(decryptedtext)

b'MyWeights'


## sending model weights

### PyTorch model defination

In [3]:
# First pyTorch model (Linear regression)
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1,
                                                requires_grad=True,
                                                dtype=torch.float))
        self.bias = nn.Parameter(torch.randn(1,
                                             requires_grad=True,
                                             dtype=torch.float))
    # To define computation in the model
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.weights * x + self.bias

In [4]:
# Create a random seed
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)

model = LinearRegressionModel()

# list(model.parameters())
dictionary = model.state_dict()
keys = dictionary.keys()
print(dictionary)
print(len(dictionary))

OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])
2


In [5]:
host = socket.gethostbyname(socket.gethostname())
port = 10080
print(host)

192.168.0.120


### setting the socket for networking

In [6]:
client_soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_soc.connect((host, port))

In [7]:
def send_msg(sock, msg, encrypt=True):
    # prefix each message with a 4-byte length in network byte order
    msg = pickle.dumps(msg)
    
    if encrypt == True:
        # instantiating the crypto engine
        engine = AES.new(key, AES.MODE_EAX, nonce)
        msg = engine.encrypt(msg)
    
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

In [8]:
send_msg(client_soc, dictionary)
client_soc.close()

In [16]:
num = 745
header = struct.pack('>I', num)
print(f"header: {header}")

header: b'\x00\x00\x02\xe9'


In [17]:
unpacked_header = struct.unpack('>I', header)[0]
print(f"unpacked header: {unpacked_header}")

unpacked header: 745


In [ ]:
# converting to bytes for encrypting
bytes_dict = pickle.dumps(dictionary)
print(f"byte_dict: {bytes_dict}\n type: {type(bytes_dict)}, size: {str(len(bytes_dict))}")

# instantiating the crypto engine
engine = AES.new(key, AES.MODE_EAX, nonce)
encrypted_dict = engine.encrypt(bytes_dict)

print(f"encrypted_byte_dict: {encrypted_dict}\n type:{type(encrypted_dict)}, , size: {str(len(encrypted_dict))}")